# Data cleaning

## Libraries used 

In [1]:
import pandas as pd
import os

## Loading data

In [2]:
data_filepath = os.path.join('data','globalterrorismdb_0617dist.xlsx')

data = pd.read_excel(data_filepath)

In [3]:
columns_to_drop = [
    'approxdate',
    'extended',
    'resolution',
    'summary',
    'crit1',
    'crit2',
    'crit3',
    'doubtterr',
    'alternative',
    'alternative_txt',
    'multiple',
    'related',
    'provstate',
    #'city',
    'vicinity',
    'location',
    'specificity',
    'success',
    'weapsubtype1',
    'weapsubtype1_txt',
    'weapsubtype2',
    'weapsubtype2_txt',
    'weapsubtype3',
    'weapsubtype3_txt',
    'weapsubtype4',
    'weapsubtype4_txt',
    'corp1',
    'target1',
    'natlty1',
    'natlty1_txt',
    'corp2',
    'target2',
    'natlty2',
    'natlty2_txt',
    'corp3',
    'target3',
    'natlty3',
    'natlty3_txt',
    'gsubname',
    'gsubname2',
    'gsubname3',
    'individual',
    'nperps',
    'nperpcap',
    'claimmode',
    'claimmode_txt',
    'compclaim',
    'claimmode2',
    'claimmode2_txt',
    'claimmode3',
    'claimmode3_txt',
    'nkillus',
    'nkillter',
    #'nwouldus',
    'nwoundte',
    'property',
    'propextent',
    'propextent_txt',
    'propvalue',
    'propcomment',
    'ishostkid',
    'nhostkid',
    #'ishostkidus',
    'nhostkidus',
    'nhours',
    'ndays',
    'divert',
    'kidhijcountry',
    'ransom',
    'ransomamt',
    #'ransomus',
    'ransomamtus',
    'ransompaid',
    'ransomnote',
    'hostkidoutcome',
    'hostkidoutcome_txt',
    'nreleased',
    'addnotes',
    'INT_LOG',
    'INT_IDEO',
    'INT_MISC',
    'INT_ANY',
    'scite1',
    'scite2',
    'scite3',
    'dbsource'
]

In [4]:
columns_to_keep = [
    'eventid',
    'iyear',
    'imonth',
    'iday',
    'country',
    'country_txt',
    'region',
    'region_txt',
    'latitude',
    'longitude',
    'attacktype1',
    'attacktype1_txt',
    'attacktype2',
    'attacktype2_txt',
    'attacktype3',
    'attacktype3_txt',
    'suicide',
    'weaptype1',
    'weaptype1_txt',
    'weaptype2',
    'weaptype2_txt',
    'weaptype3',
    'weaptype3_txt',
    'weaptype4',
    'weaptype4_txt',
    'targtype1',
    'targtype1_txt',
    'targtype2',
    'targtype2_txt',
    'targtype3',
    'targtype3_txt',
    'gname',
    'gname2',
    'gname3',
    'nkill',
    'nwould',
    
]

In [5]:
columns_dropped = data.columns.difference(columns_to_keep)
data.drop(columns_dropped, axis=1, inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170350 entries, 0 to 170349
Data columns (total 35 columns):
eventid            170350 non-null int64
iyear              170350 non-null int64
imonth             170350 non-null int64
iday               170350 non-null int64
country            170350 non-null int64
country_txt        170350 non-null object
region             170350 non-null int64
region_txt         170350 non-null object
latitude           165744 non-null float64
longitude          165744 non-null float64
suicide            170350 non-null int64
attacktype1        170350 non-null int64
attacktype1_txt    170350 non-null object
attacktype2        5630 non-null float64
attacktype2_txt    5630 non-null object
attacktype3        374 non-null float64
attacktype3_txt    374 non-null object
targtype1          170350 non-null int64
targtype1_txt      170350 non-null object
targtype2          10018 non-null float64
targtype2_txt      10018 non-null object
targtype3          1034

### Looking at the data

From the info above, we can observe that not all the columns in our data are populated for each data entry. One set of columns that are really important for our visualization, latitude and longitude, is found between in columns with missing values. Given that our project has a big focus on the geo-location dimmension of the database, a decision has to be made.

Possible solutions:
* observing that for these data entries, the country is specified, we can infer approximate values for the latitude and longitude
* looking at the distribution by year and by country of the data, we can see that the number of data entries with missing localization data is rather small comparing to the number of data entries containing geolocalization. Therefore dropping those entries would not have a big impact  


In [7]:
# we set this in order to avoid truncated dataframes when printed
pd.options.display.max_rows = 999

In order to back-up the first possible solution, we look at the number of data entries with missing values. Then we check if every data entry that is missing the geolocation has the country specified. We do this by grouping by country and sum the counts. The resulted number should be equal to the number of data entries with missing geolocation.

In [8]:
len(data[data.latitude.isnull()])

4606

In [9]:
data[data.latitude.isnull()].groupby('country_txt').count()['eventid'].sum()

4606

We can confirm that every data entry with missing geolocation has the country specified.

**Second solution**: We now try to gather more insight into the other possible solution: dropping the problematic entries.

For this, we want to know what fraction of the data has missing geolocation, considering first by country and then by year.

In [10]:
full_country_counts = data['country_txt'].value_counts()
missing_country_counts = data[data.latitude.isnull()]['country_txt'].value_counts()

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
missing_fraction_country = missing_country_counts.divide(full_country_counts, fill_value=0)
missing_fraction_country

Afghanistan                         0.007695
Albania                             0.012658
Algeria                             0.048736
Andorra                             1.000000
Angola                              0.068966
Antigua and Barbuda                 0.000000
Argentina                           0.017370
Armenia                             0.000000
Australia                           0.000000
Austria                             0.009174
Azerbaijan                          0.063830
Bahamas                             0.000000
Bahrain                             0.043011
Bangladesh                          0.043614
Barbados                            0.000000
Belarus                             0.000000
Belgium                             0.000000
Belize                              0.125000
Benin                               0.125000
Bhutan                              0.166667
Bolivia                             0.031847
Bosnia-Herzegovina                  0.006289
Botswana  

At a first glance, the number of data entries with missing geolocation is rather small comparing with the total number of entries. 
Which mean that dropping this entries would not affect out visualization.
But we will deep further do identify if there are cases where the entries with missing geolocation account for a big part of the data.

In [13]:
significant_missing = missing_fraction_country[missing_fraction_country >= 0.5]
significant_missing

Andorra                1.0
Equatorial Guinea      0.5
South Yemen            0.5
St. Kitts and Nevis    0.5
Name: country_txt, dtype: float64

We see that for three countries half of the data is missing the geolocation and all data entries for Andorra. Thinking about completing the data by hand, we will now see how many cases are for each country.

In [14]:
missing_country_counts.loc[significant_missing.index]

Andorra                1
Equatorial Guinea      1
South Yemen            1
St. Kitts and Nevis    1
Name: country_txt, dtype: int64

There is only one entry per country, therefore we can put this data in by hand.

We can see below the details about these attacks in order to Google search a more precise location.

In [15]:
data[data.country_txt.isin(significant_missing.index)].query('latitude != latitude')

,eventid,iyear,imonth,iday,country,country_txt,region,region_txt,latitude,longitude,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targtype2,targtype2_txt,targtype3,targtype3_txt,gname,gname2,gname3,weaptype1,weaptype1_txt,weaptype2,weaptype2_txt,weaptype3,weaptype3_txt,weaptype4,weaptype4_txt,nkill
2397,197407140002,1974,7,14,7,Andorra,8,Western Europe,NaN,NaN,0,2,Armed Assault,NaN,NaN,NaN,NaN,1,Business,NaN,NaN,NaN,NaN,International Revolutionary Action Group (GARI),NaN,NaN,5,Firearms,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17846,198301200014,1983,1,20,406,South Yemen,10,Middle East & North Africa,NaN,NaN,0,4,Hijacking,NaN,NaN,NaN,NaN,6,Airports & Aircraft,NaN,NaN,NaN,NaN,Palestinians,NaN,NaN,5,Firearms,NaN,NaN,NaN,NaN,NaN,NaN,0.0
51181,199205040002,1992,5,4,62,Equatorial Guinea,11,Sub-Saharan Africa,NaN,NaN,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1.0
59591,199506250005,1995,6,25,189,St. Kitts and Nevis,2,Central America & Caribbean,NaN,NaN,0,2,Armed Assault,NaN,NaN,NaN,NaN,14,Private Citizens & Property,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,8,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,0.0


After multiple tries, we did not manage to find more data about these attacks on the internet. Therefore, because we want our visualization to not introduce misinformation, we will discard them except Andorra.

Given that there is just this case of terrorist attack and that the country is rather small, in order to reflect in out visualization that there was this kind of attack there, we can use the center of the country as the location for this attack.

** Missing entries distribution per year**

In [16]:
full_yearly_counts = data['iyear'].value_counts()
missing_yearly_counts = data[data.latitude.isnull()]['iyear'].value_counts()

In [17]:
missing_fraction_yearly = missing_yearly_counts.divide(full_yearly_counts, fill_value=0)
missing_fraction_yearly

1970    0.012289
1971    0.019149
1972    0.008065
1973    0.016913
1974    0.006897
1975    0.017568
1976    0.024919
1977    0.020470
1978    0.047837
1979    0.049605
1980    0.040195
1981    0.049516
1982    0.055403
1983    0.072822
1984    0.087268
1985    0.083362
1986    0.064336
1987    0.064070
1988    0.068280
1989    0.042100
1990    0.062259
1991    0.060645
1992    0.048492
1994    0.048872
1995    0.052905
1996    0.050065
1997    0.065000
1998    0.046088
1999    0.038710
2000    0.023166
2001    0.012585
2002    0.016517
2003    0.011886
2004    0.024957
2005    0.015928
2006    0.015278
2007    0.015736
2008    0.010410
2009    0.003179
2010    0.001659
2011    0.005916
2012    0.006118
2013    0.009170
2014    0.006465
2015    0.006800
2016    0.004004
Name: iyear, dtype: float64

In [18]:
missing_fraction_yearly[missing_fraction_yearly >= 0.1]

Series([], Name: iyear, dtype: float64)

We can remark that the number of data entries with missing geolocation is smaller than 10% for each year. Therefore, taking into consideration the year also, dropping these entries will not have a great impact for our visualization

## Decision

We decided to go with the second solution because infering the approximate values could create artificial patterns in our vizualisation and that would mean giving wrong information to our user. We will make an exception for Andorra where there is only one recorded attack in the entire database and so is worth infering the location from the country for this specific case.

In [19]:
andorra_lat = 42.544033
andorra_long = 1.556309
data.loc[data['country_txt']=='Andorra', 'latitude'] = andorra_lat
data.loc[data['country_txt']=='Andorra', 'longitude'] = andorra_long

We can now keep only the data entries that have geolocation data.

In [20]:
data = data[~data.latitude.isnull()]

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165745 entries, 0 to 170349
Data columns (total 35 columns):
eventid            165745 non-null int64
iyear              165745 non-null int64
imonth             165745 non-null int64
iday               165745 non-null int64
country            165745 non-null int64
country_txt        165745 non-null object
region             165745 non-null int64
region_txt         165745 non-null object
latitude           165745 non-null float64
longitude          165745 non-null float64
suicide            165745 non-null int64
attacktype1        165745 non-null int64
attacktype1_txt    165745 non-null object
attacktype2        5557 non-null float64
attacktype2_txt    5557 non-null object
attacktype3        370 non-null float64
attacktype3_txt    370 non-null object
targtype1          165745 non-null int64
targtype1_txt      165745 non-null object
targtype2          9890 non-null float64
targtype2_txt      9890 non-null object
targtype3          1011 n

Inspecting the info above about the columns data type, we can remark that the type is *object* only for columns that contain text. This mean the data is consistent in terms of values for the other columns (i.e. if integers are used as a codification, then they are used throughout all data entries) 

In [22]:
data.head()

,eventid,iyear,imonth,iday,country,country_txt,region,region_txt,latitude,longitude,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targtype2,targtype2_txt,targtype3,targtype3_txt,gname,gname2,gname3,weaptype1,weaptype1_txt,weaptype2,weaptype2_txt,weaptype3,weaptype3_txt,weaptype4,weaptype4_txt,nkill
0,197000000001,1970,7,2,58,Dominican Republic,2,Central America & Caribbean,18.456792,-69.951164,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,197000000002,1970,0,0,130,Mexico,1,North America,19.432608,-99.133207,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),NaN,NaN,NaN,NaN,23rd of September Communist League,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,197001000001,1970,1,0,160,Philippines,5,Southeast Asia,15.478598,120.599741,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,197001000002,1970,1,0,78,Greece,8,Western Europe,37.983773,23.728157,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),NaN,NaN,NaN,NaN,Unknown,NaN,NaN,6,Explosives/Bombs/Dynamite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,197001000003,1970,1,0,101,Japan,4,East Asia,33.580412,130.396361,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,7,Government (Diplomatic),NaN,NaN,NaN,NaN,Unknown,NaN,NaN,8,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As discussed in the project book,  we will keep certain columns are reference. For this, we will create a dictionary containing the columns with their conventions which we will later save as json.

In [23]:
refs_dict = {}

In [24]:
refs_dict['attacktype'] = dict(zip(data['attacktype1'].astype(str), data['attacktype1_txt']))
refs_dict['attacktype']

{'1': 'Assassination',
 '2': 'Armed Assault',
 '3': 'Bombing/Explosion',
 '4': 'Hijacking',
 '5': 'Hostage Taking (Barricade Incident)',
 '6': 'Hostage Taking (Kidnapping)',
 '7': 'Facility/Infrastructure Attack',
 '8': 'Unarmed Assault',
 '9': 'Unknown'}

In [25]:
refs_dict['targtype'] = dict(zip(data['targtype1'].astype(str), data['targtype1_txt']))
refs_dict['targtype']

{'1': 'Business',
 '10': 'Journalists & Media',
 '11': 'Maritime',
 '12': 'NGO',
 '13': 'Other',
 '14': 'Private Citizens & Property',
 '15': 'Religious Figures/Institutions',
 '16': 'Telecommunication',
 '17': 'Terrorists/Non-State Militia',
 '18': 'Tourists',
 '19': 'Transportation',
 '2': 'Government (General)',
 '20': 'Unknown',
 '21': 'Utilities',
 '22': 'Violent Political Party',
 '3': 'Police',
 '4': 'Military',
 '5': 'Abortion Related',
 '6': 'Airports & Aircraft',
 '7': 'Government (Diplomatic)',
 '8': 'Educational Institution',
 '9': 'Food or Water Supply'}

In [26]:
if 'targsubtype1' in data.columns:
    targsubtype_notnull = data[data.targsubtype1.notnull()]
    reft_dict['targsubtype'] = dict(zip(targsubtype_notnull['targsubtype1'].astype(str),\
                                        targsubtype_notnull['targsubtype1_txt'])) 

In [27]:
refs_dict['weaptype'] = dict(zip(data['weaptype1'].astype(str), data['weaptype1_txt']))
if '4' not in refs_dict['weaptype']:
    refs_dict['weaptype']['4']='Nuclear'
refs_dict['weaptype']

{'1': 'Biological',
 '10': 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)',
 '11': 'Sabotage Equipment',
 '12': 'Other',
 '13': 'Unknown',
 '2': 'Chemical',
 '3': 'Radiological',
 '4': 'Nuclear',
 '5': 'Firearms',
 '6': 'Explosives/Bombs/Dynamite',
 '7': 'Fake Weapons',
 '8': 'Incendiary',
 '9': 'Melee'}

In [28]:
refs_dict['country'] = dict(zip(data['country'].astype(str), data['country_txt']))
refs_dict['country']

{'10': 'Antigua and Barbuda',
 '100': 'Jamaica',
 '1001': 'Serbia',
 '1002': 'Montenegro',
 '1003': 'Kosovo',
 '1004': 'South Sudan',
 '101': 'Japan',
 '102': 'Jordan',
 '103': 'Kazakhstan',
 '104': 'Kenya',
 '106': 'Kuwait',
 '107': 'Kyrgyzstan',
 '108': 'Laos',
 '109': 'Latvia',
 '11': 'Argentina',
 '110': 'Lebanon',
 '111': 'Lesotho',
 '112': 'Liberia',
 '113': 'Libya',
 '115': 'Lithuania',
 '116': 'Luxembourg',
 '117': 'Macau',
 '118': 'Macedonia',
 '119': 'Madagascar',
 '12': 'Armenia',
 '120': 'Malawi',
 '121': 'Malaysia',
 '122': 'Maldives',
 '123': 'Mali',
 '124': 'Malta',
 '127': 'Martinique',
 '128': 'Mauritania',
 '129': 'Mauritius',
 '130': 'Mexico',
 '132': 'Moldova',
 '136': 'Morocco',
 '137': 'Mozambique',
 '138': 'Myanmar',
 '139': 'Namibia',
 '14': 'Australia',
 '141': 'Nepal',
 '142': 'Netherlands',
 '143': 'New Caledonia',
 '144': 'New Zealand',
 '145': 'Nicaragua',
 '146': 'Niger',
 '147': 'Nigeria',
 '149': 'North Korea',
 '15': 'Austria',
 '151': 'Norway',
 '153':

In [29]:
refs_dict['region'] = dict(zip(data['region'].astype(str), data['region_txt']))
refs_dict['region']

{'1': 'North America',
 '10': 'Middle East & North Africa',
 '11': 'Sub-Saharan Africa',
 '12': 'Australasia & Oceania',
 '2': 'Central America & Caribbean',
 '3': 'South America',
 '4': 'East Asia',
 '5': 'Southeast Asia',
 '6': 'South Asia',
 '7': 'Central Asia',
 '8': 'Western Europe',
 '9': 'Eastern Europe'}

We can now drop the textual columns because we created the dictionary with their value.

In [30]:
data.columns[data.columns.str.endswith('_txt')]

Index(['country_txt', 'region_txt', 'attacktype1_txt', 'attacktype2_txt',
       'attacktype3_txt', 'targtype1_txt', 'targtype2_txt', 'targtype3_txt',
       'weaptype1_txt', 'weaptype2_txt', 'weaptype3_txt', 'weaptype4_txt'],
      dtype='object')

In [31]:
textual_columns_to_drop = data.columns[data.columns.str.endswith('_txt')] 
data = data.drop(textual_columns_to_drop,axis=1)

Now we can dump the dataframe to a json file. A thing to notice here is because we wanted the json file the have a size as small as possible, we chose the **orient** option to be **"values"**. This means that the json file will contain an array of arrays (a matrix), where each row is a data entry. We will also create a json file with the columns so that we know what the columns represent

For space reasons, we will also merge columns like **attacktype1 attacktype2 attacktype3** into a single column containing an array where the null value will be ignored.

In [32]:
def merge_column(row, base_column_name, number_of_columns):
    res = []
    for i in range(1,number_of_columns + 1):
        col_name = base_column_name + str(i)
        if row[col_name] == row[col_name]:
            res.append(row[col_name])
    return tuple(res)

In [33]:
data.rename(columns={'gname':'gname1'}, inplace=True)

data['attacktype'], data['targtype'], data['weaptype'], data['gname'] = \
    zip(*data.apply(lambda row: (merge_column(row, 'attacktype', 3),
                               merge_column(row, 'targtype', 3),
                               merge_column(row, 'weaptype', 4),
                               merge_column(row, 'gname', 3)), 
                   axis=1
                  )
       )

In [34]:
cols_to_drop = ['attacktype1', 'attacktype2','attacktype3','targtype1','targtype2','targtype3',
               'weaptype1','weaptype2','weaptype3','weaptype4','gname1','gname2','gname3']

data.drop(cols_to_drop, axis=1, inplace=True)

In [35]:
columns_dict = {}
for col, idx in zip(data.columns.astype(str), range(len(data.columns))):
    columns_dict[col] = idx

# we create a mapping for both ways: from column names to indices and from indices to column names
refs_dict['columns'] = {**{str(v): k for k, v in columns_dict.items()}, **columns_dict}

Another thing we noticed in our data is that the **nkill** columns cand have null values as result of uncertain reports. In this matter, we follow the GTD methodology to take the lowest number in case of multiple reports, hence here we will asume 0 as there are none.

In [68]:
data.fillna(value={'nkill':0}, inplace=True)

,eventid,iyear,imonth,iday,country,region,latitude,longitude,suicide,nkill,attacktype,targtype,weaptype,gname
0,197000000001,1970,7,2,58,2,18.456792,-69.951164,0,1.0,"(1,)","(14,)","(13,)","(MANO-D,)"
1,197000000002,1970,0,0,130,1,19.432608,-99.133207,0,0.0,"(6,)","(7,)","(13,)","(23rd of September Communist League,)"
2,197001000001,1970,1,0,160,5,15.478598,120.599741,0,1.0,"(1,)","(10,)","(13,)","(Unknown,)"
3,197001000002,1970,1,0,78,8,37.983773,23.728157,0,0.0,"(3,)","(7,)","(6,)","(Unknown,)"
4,197001000003,1970,1,0,101,4,33.580412,130.396361,0,0.0,"(7,)","(7,)","(8,)","(Unknown,)"
5,197001010002,1970,1,1,217,1,37.005105,-89.176269,0,0.0,"(2,)","(3,)","(5,)","(Black Nationalists,)"
6,197001020001,1970,1,2,218,3,-34.891151,-56.187214,0,0.0,"(1,)","(3,)","(5,)","(Tupamaros (Uruguay),)"
7,197001020002,1970,1,2,217,1,37.805065,-122.273024,0,0.0,"(3,)","(21,)","(6,)","(Unknown,)"
8,197001020003,1970,1,2,217,1,43.076592,-89.412488,0,0.0,"(7,)","(4,)","(8,)","(New Year's Gang,)"
9,197001030001,1970,1,3,217,1,43.072950,-89.386694,0,0.0,"(7,)","(2,)","(8,)","(New Year's Gang,)"


In [69]:
import json

events_list = json.loads(data.to_json(orient='values'))
final_data = {"refs":refs_dict,
             "events":events_list}

In [70]:
with open('db.json', 'w') as fp:
    json.dump(final_data, fp)